In [4]:
import cv2
import math
import os
os.chdir(r'D:\Python37\Projects\Gender-and-Age-Detection-Mona\Gender-and-Age-Detection\models')

def highlightFace(net, frame, conf_threshold=0.7):
    frameOpencvDnn=frame.copy()
    frameHeight=frameOpencvDnn.shape[0]
    frameWidth=frameOpencvDnn.shape[1]
    blob=cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)
    net.setInput(blob)
    detections=net.forward()
    faceBoxes=[]
    for i in range(detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>conf_threshold:
            x1=int(detections[0,0,i,3]*frameWidth)
            y1=int(detections[0,0,i,4]*frameHeight)
            x2=int(detections[0,0,i,5]*frameWidth)
            y2=int(detections[0,0,i,6]*frameHeight)
            faceBoxes.append([x1,y1,x2,y2])
            cv2.rectangle(frameOpencvDnn, (x1,y1), (x2,y2), (0,255,0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn,faceBoxes

faceProto="opencv_face_detector.pbtxt"
faceModel="opencv_face_detector_uint8.pb"
ageProto="age_deploy.prototxt"
ageModel="age_net.caffemodel"
genderProto="gender_deploy.prototxt"
genderModel="gender_net.caffemodel"

MODEL_MEAN_VALUES=(103.93, 116.77, 123.68)
ageList=['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList=['Male','Female']

faceNet=cv2.dnn.readNet(faceModel,faceProto)

ageNet=cv2.dnn.readNet(ageModel,ageProto)
genderNet=cv2.dnn.readNet(genderModel,genderProto)

video=cv2.VideoCapture(0)
padding=20
while cv2.waitKey(1)<0 :
    hasFrame,frame=video.read()
    if not hasFrame:
        cv2.waitKey()
        break
    
    resultImg,faceBoxes=highlightFace(faceNet,frame)
    if not faceBoxes:
        print("No face detected")

    for faceBox in faceBoxes:
        face=frame[max(0,faceBox[1]-padding):min(faceBox[3]+padding,frame.shape[0]-1),max(0,faceBox[0]-padding):min(faceBox[2]+padding, frame.shape[1]-1)]
        print(face)
        blob=cv2.dnn.blobFromImage(face, 1.0, (227,227), MODEL_MEAN_  VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds=genderNet.forward()
        gender=genderList[genderPreds[0].argmax()]
        #print(f'Gender: {gender}')
        #print(genderPreds)

        ageNet.setInput(blob)
        agePreds=ageNet.forward()
        age=ageList[agePreds[0].argmax()]
        #print(f'Age: {age[1:-1]} years')

        cv2.putText(resultImg, f'{gender}, {age}', (faceBox[0], faceBox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2, cv2.LINE_AA)
        cv2.imshow("Detecting age and gender", resultImg)
        
        if cv2.waitKey(33) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()      


[[[253 255 253]
  [253 255 253]
  [253 255 253]
  ...
  [218 238 246]
  [218 238 246]
  [218 238 246]]

 [[253 255 253]
  [253 255 253]
  [253 255 253]
  ...
  [218 238 246]
  [219 239 247]
  [219 239 247]]

 [[251 255 253]
  [251 255 253]
  [251 255 253]
  ...
  [220 239 247]
  [222 241 249]
  [223 242 250]]

 ...

 [[ 43  26  20]
  [ 41  26  20]
  [ 42  27  21]
  ...
  [ 38  18  10]
  [ 39  20  10]
  [ 38  19   9]]

 [[ 43  26  20]
  [ 40  25  19]
  [ 41  26  20]
  ...
  [ 36  16   8]
  [ 37  18   8]
  [ 37  18   8]]

 [[ 41  24  18]
  [ 37  22  16]
  [ 38  23  17]
  ...
  [ 36  16  10]
  [ 37  17   9]
  [ 37  17   9]]]
[[[253 255 253]
  [253 255 253]
  [253 255 253]
  ...
  [218 234 244]
  [220 236 246]
  [222 238 248]]

 [[253 255 253]
  [253 255 253]
  [253 255 253]
  ...
  [218 234 244]
  [218 234 244]
  [222 238 248]]

 [[253 255 253]
  [253 255 253]
  [253 255 253]
  ...
  [218 236 246]
  [219 237 247]
  [222 240 250]]

 ...

 [[ 39  27  18]
  [ 39  27  20]
  [ 36  25  20]
  ..